In [1]:
import os
os.getcwd()

'/home/jovyan/ArditArifi'

In [2]:
source_path="/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6"
output_path="/home/jovyan/student-storages/GROUP3/ArditArifi/output"

! mkdir -p output_path

Analysis

In [3]:
import glob
# ANALYSIS
import xarray as xr
import pandas as p
import numpy as np
# PLOTTING
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors


In [4]:

# REGIONS
region_label = ["ARC", "ANT"]
arctic_lat = [60, 90]
antarctic_lat = [-90, -60]

# SEASONS
seasons_label = ["DJF", "MAM", "JJA", "SON"]
seasons = {
    "DJF": [12, 1, 2],   # Dec, Jan, Feb
    "MAM": [3, 4, 5],    # Mar, Apr, May
    "JJA": [6, 7, 8],    # Jun, Jul, Aug
    "SON": [9, 10, 11],  # Sep, Oct, Nov
}

# Create combined region-season labels
region_season_label = [f"{region} {season}" for region in region_label for season in seasons_label]

# EXPERIMENTS
experiments = ["control", "2xss", "2xdust", "2xfire", "2xDMS"]

# MODELS
models = ["IPSL-CM6A-LR-INCA", "UKESM1-0-LL", "NorESM2-LM"]

# PATTERN FUNCTION
patter = lambda model, var, exp: f"{source_path}/{var}*{model}*piClim-{exp}*.nc"

In [11]:
# CONSTRUCT TABLE
mean_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)
std_values = np.full(
    shape=(len(region_season_label), len(experiments), len(models)),
    fill_value=np.nan
)


# LOAD DATA
model = ""
var = "rtmt"

# Calculate total steps for progress
total_steps = len(models) * len(experiments) * len(seasons_label)
current_step = 0

# MAIN LOOP
for idx_model, model in enumerate(models):
    for idx_exp, exp in enumerate(experiments):
        ds = None
        try:
            # Attempt to open the dataset
            ds = xr.open_mfdataset(patter(model, var, exp), combine="by_coords")
            # 1) Load dataset for this model/var/exp
        except Exception as e:
            # If there's an error reading files, set to 0 for ARC and ANT
            print(f"Error opening files {patter(model, var, exp)}: {e}")
            current_step += len(seasons_label)
            continue               
        
        for idx_season, season in enumerate(seasons_label):
            # Increment the progress
            current_step += 1
            # Compute and display percentage
            pct = (current_step / total_steps) * 100
            print(f"Progress: {current_step}/{total_steps} -> {pct:.2f}%")
            
            # Select months for this season
            ds_season = ds.sel(time=ds.time.dt.month.isin(seasons[season]))

            # ---------------------
            # ARCTIC
            # ---------------------
            ds_season_arctic = ds_season.sel(
                lat=slice(arctic_lat[0], arctic_lat[1]),
                lon=slice(-180, 180)
            )
            weights_arctic = np.cos(np.deg2rad(ds_season_arctic.lat))

            # Weighted mean over lat/lon => time series
            ds_season_arctic_area_ts = ds_season_arctic[var].weighted(weights_arctic).mean(dim=("lat", "lon"))

            # Multi-year average (time)
            arc_mean = ds_season_arctic_area_ts.mean("time").values
            mean_values[idx_season, idx_exp, idx_model] = arc_mean

            arc_std = ds_season_arctic_area_ts.std("time").values
            std_values[idx_season, idx_exp, idx_model] = arc_std
            
            # ---------------------
            # ANTARCTIC
            # ---------------------
            ds_season_antarctic = ds_season.sel(
                lat=slice(antarctic_lat[0], antarctic_lat[1]),
                lon=slice(-180, 180)
            )
            weights_antarctic = np.cos(np.deg2rad(ds_season_antarctic.lat))

            ds_season_antarctic_area_ts = ds_season_antarctic[var].weighted(weights_antarctic).mean(dim=("lat","lon"))

            # Multi-year average (time)
            ant_mean = ds_season_antarctic_area_ts.mean("time").values
            mean_values[idx_season + 4, idx_exp, idx_model] = ant_mean

            ant_std = ds_season_antarctic_area_ts.std("time").values
            std_values[idx_season + 4, idx_exp, idx_model] = ant_std
            
# Finally, display the populated array
print("Shape of mean_values:", mean_values.shape)
print(mean_values)

Progress: 1/60 -> 1.67%
Progress: 2/60 -> 3.33%
Progress: 3/60 -> 5.00%
Progress: 4/60 -> 6.67%
Progress: 5/60 -> 8.33%
Progress: 6/60 -> 10.00%
Progress: 7/60 -> 11.67%
Progress: 8/60 -> 13.33%
Progress: 9/60 -> 15.00%
Progress: 10/60 -> 16.67%
Progress: 11/60 -> 18.33%
Progress: 12/60 -> 20.00%
Error opening files /home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*IPSL-CM6A-LR-INCA*piClim-2xfire*.nc: no files to open
Error opening files /home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6/rtmt*IPSL-CM6A-LR-INCA*piClim-2xDMS*.nc: no files to open
Progress: 21/60 -> 35.00%
Progress: 22/60 -> 36.67%
Progress: 23/60 -> 38.33%
Progress: 24/60 -> 40.00%
Progress: 25/60 -> 41.67%
Progress: 26/60 -> 43.33%
Progress: 27/60 -> 45.00%
Progress: 28/60 -> 46.67%
Progress: 29/60 -> 48.33%
Progress: 30/60 -> 50.00%
Progress: 31/60 -> 51.67%
Progress: 32/60 -> 53.33%
Progress: 33/60 -> 55.00%
Progress: 34/60 -> 56.67%
Progress: 35/60 -> 58.33%
Progress: 36/60 -> 60.00%
Progress: 37/60 -> 61.67%
Pro

In [22]:
import pandas as pd

bias = mean_values[:, 1:, :] - mean_values[:, 0, :][:, np.newaxis, :]
for idx_exp in range(len(experiments) - 1):
    # This will skip the control experiment at index 0
    exp_name = experiments[1 + idx_exp]
    print("Experiment:", exp_name)
    
    # bias[:, idx_exp, :].shape -> (len(region_season_label), len(models))
    # bias[:, idx_exp, :].transpose().shape -> (len(models), len(region_season_label))
    # We want models as rows and region_season_label as columns
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    print(df)
    print()  # Blank line for readability

Experiment: 2xss
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.157379 -0.600880 -1.143471 -0.077911 -0.637404 -0.164734   
UKESM1-0-LL       -0.260221  0.168703 -0.497273 -0.050565 -0.399657 -0.295128   
NorESM2-LM        -0.802078 -0.343978 -0.872966 -0.441364 -1.438295 -0.408204   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.064667 -0.080124  
UKESM1-0-LL        0.103706 -0.346260  
NorESM2-LM        -0.474610 -0.778683  

Experiment: 2xdust
                    ARC DJF   ARC MAM   ARC JJA   ARC SON   ANT DJF   ANT MAM  \
IPSL-CM6A-LR-INCA  0.557541 -0.596718 -0.590376  0.374619 -0.292732  0.030563   
UKESM1-0-LL       -0.096941  0.280963 -0.422601  0.101109 -0.039652 -0.178474   
NorESM2-LM         0.089839  0.680931  0.548995  0.042616 -0.334020  0.565876   

                    ANT JJA   ANT SON  
IPSL-CM6A-LR-INCA -0.249527 -0.071815  
UKESM1-0-LL        0.129572  0.117032  
NorESM2-LM         1.228110  0.92

In [23]:
import pandas as pd
import numpy as np

# Suppose the below variables are already defined
# bias : shape (len(region_season_label), len(experiments)-1, len(models))
# region_season_label : list of region/season names
# experiments : list of experiments, with experiment[0] = "control"
# models : list of model names

all_data = []  # Will hold the melted DataFrame for each experiment

for idx_exp in range(len(experiments) - 1):
    exp_name = experiments[idx_exp + 1]
    print("Experiment:", exp_name)

    # bias[:, idx_exp, :].shape -> (n_region_season, n_models)
    # transpose -> (n_models, n_region_season)
    df = pd.DataFrame(
        data=bias[:, idx_exp, :].transpose(),
        index=models,
        columns=region_season_label
    )

    # Reshape df to "long" format with columns [Model, region_season_label, value]
    df_long = df.reset_index().melt(
        id_vars="index",                 # "index" column is the model
        var_name="RegionSeason",         # name for the melted columns
        value_name="Bias"               # name for the melted values
    )

    # Rename "index" -> "Model"
    df_long.rename(columns={"index": "Model"}, inplace=True)

    # Add a column for Experiment
    df_long["Experiment"] = exp_name

    # Append to the list
    all_data.append(df_long)

# Concatenate all experiments into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Optionally reorder columns (Experiment, Model, RegionSeason, Bias)
final_df = final_df[["Experiment", "Model", "RegionSeason", "Bias"]]

# Save to CSV
final_df.to_csv("rtmts.csv", index=False)

print("CSV file saved: all_bias_results.csv")


Experiment: 2xss
Experiment: 2xdust
Experiment: 2xfire
Experiment: 2xDMS
CSV file saved: all_bias_results.csv
